In [1]:
import empyrical as ep
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_ta as ta
import pyfolio as pf
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import yfinance as yf

import datetime
import warnings
import pytz

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier

from xgboost import XGBClassifier

c:\Users\HP Victus 16\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [2]:
plt.style.use("tableau-colorblind10")
# Disable future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

yf.pdr_override()

In [3]:
tickers = [
    "AAV", "ADVANC", "AEONTS", "AMATA", "AOT", "AP", "AURA", "AWC", "BAM", "BANPU",
    "BBL", "BCH", "BCP", "BCPG", "BDMS", "BEM", "BGRIM", "BH", "BLA", 
    # "BSRC",
    "BTG", "BTS", "BYD", "CBG", "CENTEL", "CHG", "CK", "COM7", "CPALL", "CPF",
    "CPN", "CRC", "DELTA", "DOHOME", "EA", "EGCO", "ERW", "FORTH", "GLOBAL", "GPSC",
    "GULF", "GUNKUL", "HANA", "HMPRO", "ICHI", "INTUCH", "IRPC", "ITC", "IVL", "JMART",
    "JMT", "KBANK", "KCE", "KKP", "KTB", "KTC", "LH", "M", "MEGA", "MINT",
    "MOSHI", "MTC", "NEX", "OR", "ORI", "OSP", "PLANB", "PTT", "PTTEP", "PTTGC",
    "RATCH", "RBF", "RCL", "SAPPE", "SAWAD", "SCB", "SCC", "SCGP", "SIRI", "SISB",
    "SJWD", "SNNP", "SPALI", "SPRC", "STA", "STGT", "TASCO", "TCAP", "THG", "TIDLOR",
    "TISCO", "TKN", "TLI", "TOA", "TOP", "TRUE", "TTB", "TU", "VGI", "WHA"
]

In [4]:
tickers_list = [ ticker + ".BK" for ticker in tickers ]
tickers_list.insert(0,"TDEX.BK")
tickers_list.insert(0,"^SET.BK")
tickers_list

['^SET.BK',
 'TDEX.BK',
 'AAV.BK',
 'ADVANC.BK',
 'AEONTS.BK',
 'AMATA.BK',
 'AOT.BK',
 'AP.BK',
 'AURA.BK',
 'AWC.BK',
 'BAM.BK',
 'BANPU.BK',
 'BBL.BK',
 'BCH.BK',
 'BCP.BK',
 'BCPG.BK',
 'BDMS.BK',
 'BEM.BK',
 'BGRIM.BK',
 'BH.BK',
 'BLA.BK',
 'BTG.BK',
 'BTS.BK',
 'BYD.BK',
 'CBG.BK',
 'CENTEL.BK',
 'CHG.BK',
 'CK.BK',
 'COM7.BK',
 'CPALL.BK',
 'CPF.BK',
 'CPN.BK',
 'CRC.BK',
 'DELTA.BK',
 'DOHOME.BK',
 'EA.BK',
 'EGCO.BK',
 'ERW.BK',
 'FORTH.BK',
 'GLOBAL.BK',
 'GPSC.BK',
 'GULF.BK',
 'GUNKUL.BK',
 'HANA.BK',
 'HMPRO.BK',
 'ICHI.BK',
 'INTUCH.BK',
 'IRPC.BK',
 'ITC.BK',
 'IVL.BK',
 'JMART.BK',
 'JMT.BK',
 'KBANK.BK',
 'KCE.BK',
 'KKP.BK',
 'KTB.BK',
 'KTC.BK',
 'LH.BK',
 'M.BK',
 'MEGA.BK',
 'MINT.BK',
 'MOSHI.BK',
 'MTC.BK',
 'NEX.BK',
 'OR.BK',
 'ORI.BK',
 'OSP.BK',
 'PLANB.BK',
 'PTT.BK',
 'PTTEP.BK',
 'PTTGC.BK',
 'RATCH.BK',
 'RBF.BK',
 'RCL.BK',
 'SAPPE.BK',
 'SAWAD.BK',
 'SCB.BK',
 'SCC.BK',
 'SCGP.BK',
 'SIRI.BK',
 'SISB.BK',
 'SJWD.BK',
 'SNNP.BK',
 'SPALI.BK',
 'SPRC.BK'

In [5]:
def fetch_data(ticker, start, end):
    df = yf.download(ticker, start=start, end=end)
    df.index = pd.to_datetime(df.index)  # Ensure index is datetime
    return df

In [6]:
def detect_trends(df, max_drawdown=30):
    
    trends = np.empty((0, 3), np.datetime64)
    x = len(df)
    trend = ''
    Peak = -np.inf
    date_Peak = None
    Trough = np.inf
    date_Trough = None

    for i in range(x):
        
        up = 0
        dn = 0
        
        if trend == '' or trend == 'bull':
            if df.iloc[i]['Close'] >= Peak:
                Peak = df.iloc[i]['Close']
                date_Peak = df.index[i]
                
        if trend == '' or trend == 'bear':
            if df.iloc[i]['Close'] <= Trough:
                Trough = df.iloc[i]['Close']
                date_Trough = df.index[i]
                
        if Peak != -np.inf:
            dn = (Peak - df.iloc[i]['Close']) / (Peak / 100.0)
            
        if Trough != np.inf:
            up = (df.iloc[i]['Close'] - Trough) / (Trough / 100.0)

        if up >= max_drawdown:
            trend = 'bull'
            trends = np.append(trends, np.array([[date_Trough, df.index[i], 1]]), axis=0)
            Trough = np.inf
            Peak = df.iloc[i]['Close']
            date_Peak = df.index[i]

        if dn >= max_drawdown:
            trend = 'bear'
            trends = np.append(trends, np.array([[date_Peak, df.index[i], 2]]), axis=0)
            Peak = -np.inf
            Trough = df.iloc[i]['Close']
            date_Trough = df.index[i]

    return trends

In [7]:
def label_trends(df, trends):
    df['Trend'] = 0  # Initialize trend column with 0
    for trend in trends:
        start_date = trend[0]
        end_date = trend[1]
        trend_type = trend[2]
        df.loc[(df.index >= start_date) & (df.index <= end_date), 'Trend'] = trend_type

    return df


In [8]:
startDate = "1990-01-01"
endDate = "2024-01-01"

all_data = pd.DataFrame()

showModelScores = False
gridSearch = False
cvTest = False

# 
# tickers_list = ['^SET.BK', 'DELTA.BK']

for ticker in tickers_list:
    
    dft = yf.Ticker(ticker)
    timeZone = dft.info.get("timeZoneFullName")
    tickerName = dft.info.get("longName", "Unknown Ticker")
    
    df = fetch_data(ticker, startDate, endDate)
    
    df['MACD'] = ta.macd(df['Close'], fast=12, slow=26, signal=9)['MACD_12_26_9']
    df['Pct_Change'] = df['Close'].pct_change() * 100

    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))

    avg_gain = gain.ewm(span=14, min_periods=14).mean()
    avg_loss = loss.ewm(span=14, min_periods=14).mean()
    rs = avg_gain / avg_loss
    df['RSI_EMA'] = 100 - (100 / (1 + rs))

    df['RSI_ta'] = ta.rsi(df['Close'], length=14)
    df['MA10'] = df.ta.sma(length=10)
    df['MA50'] = df.ta.sma(length=50)
    df['MA200'] = df.ta.sma(length=200)

    window = 20
    df['MA20'] = df['Close'].rolling(window=window).mean()
    df['std_dev'] = df['Close'].rolling(window=window).std()
    df['Upper_BB'] = df['MA20'] + (df['std_dev'] * 2)
    df['Lower_BB'] = df['MA20'] - (df['std_dev'] * 2)
    
    df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
    
    df = df.dropna()
    
    trends = detect_trends(df)
    df = label_trends(df, trends)
        
    vix = yf.Ticker("^VIX")
    vix_df = vix.history(interval="1d", start=startDate, end=endDate)
    
    # Calculate percentage change for VIX
    # Convert vix_df index timezone to match df
    stock_timezone = pytz.timezone(timeZone)
    vix_df.index = vix_df.index.tz_convert(stock_timezone)
    vix_df.index = vix_df.index.tz_localize(None)  # Convert to naive timezone

    # Ensure matching datetime indices and reindex vix_df
    vix_df = vix_df.reindex(df.index, method='ffill')

    # Merge VIX close prices into the original DataFrame
    df['VIX'] = vix_df['Close']
    
    X = df[['Pct_Change', 'std_dev', 'MA200', 'MA50', 'MA10', 'MACD', 'RSI_ta', 'VIX']]
    y = df['Target']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    rf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
    xgb = XGBClassifier(objective='binary:logistic', colsample_bytree=0.8, gamma=0.1, learning_rate=0.001, max_depth=3, n_estimators=100, reg_alpha=0.1, reg_lambda=0.1, subsample=0.8)
    
    # rf.fit(X_train, y_train)
    # xgb.fit(X_train, y_train)
    
    model = VotingClassifier(estimators=[('rf', rf), ('xgb', xgb)], voting='soft')
    model.fit(X_train, y_train)

    if showModelScores:
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        print(f"Accuracy of the model: {acc:.4f}")

        cm = confusion_matrix(y_test, y_pred)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm)
        disp.plot()
        plt.show()
    
    df['Predict'] = 0
    
    df.loc[X_test.index, 'Predict'] = model.predict(X_test)
    
    df['Position'] = 0
    
    start_idx = df.index.get_loc(X_test.index[0])

    positions = []
    
    holding_position = False
    for i in range(start_idx, len(df)):
        if df.loc[df.index[i], 'Trend'] == 1 and df.loc[df.index[i], 'Predict'] == 1 and not holding_position:
            df.loc[df.index[i], 'Position'] = 1
            holding_position = True
            positions.append(df.index[i])
        elif df.loc[df.index[i], 'Trend'] == 2 or df.loc[df.index[i], 'Predict'] == 0 and holding_position:
            df.loc[df.index[i], 'Position'] = 0
            holding_position = False

    
    df.loc[df['Position'] == 0, 'Position'] = np.nan
    df['Position'] = df['Position'].fillna(method='ffill')
    df['Position'] = df['Position'].fillna(0).astype(int)

    df_test_period = df.loc[X_test.index[0]:]
    df_test_period = df_test_period.copy()
    df_test_period['Strategy_Returns'] = df_test_period['Position'].shift(1) * df_test_period['Close'].pct_change()
    df_test_period.dropna(subset=['Strategy_Returns'], inplace=True)

    returns = df_test_period['Strategy_Returns']
    
    cumulative_returns = (1 + returns).cumprod() - 1
    perf_stats = {
        'Annual Return': ep.annual_return(returns),
        'Cumulative Returns': ep.cum_returns_final(returns),
        'Annual Volatility': ep.annual_volatility(returns),
        'Sharpe Ratio': ep.sharpe_ratio(returns),
        'Sortino Ratio': ep.sortino_ratio(returns),
        'Max Drawdown': ep.max_drawdown(returns),
        'Calmar Ratio': ep.calmar_ratio(returns)
    }
    
    perf_df = pd.DataFrame(perf_stats, index=[ticker])
    
    perf_df['Index Name'] = tickerName
    
    perf_df['1_Train'] = model.score(X_train,y_train)
    perf_df['1_Test'] = model.score(X_test,y_test)
    
    perf_df['Test Date'] = str(X_test.index[0])
    
    perf_df = perf_df[['Index Name'] + [col for col in perf_df.columns if col != 'Index Name']]
    
    # Append to all_data DataFrame
    all_data = pd.concat([all_data, perf_df])
    
    print(f"{ticker} | Train : {model.score(X_train, y_train):.4f} | Test : {model.score(X_test, y_test):.4f}")
    print(f"{ep.annual_return(returns):.5f} {ep.annual_volatility(returns):.5f} {ep.sharpe_ratio(returns):.5f} {ep.max_drawdown(returns):.5f}")

[*********************100%%**********************]  1 of 1 completed


^SET.BK | Train : 0.6054 | Test : 0.5051
0.06577 0.12097 0.58689 -0.20735


[*********************100%%**********************]  1 of 1 completed


TDEX.BK | Train : 0.6700 | Test : 0.4878
0.00000 0.00000 nan 0.00000


[*********************100%%**********************]  1 of 1 completed


...

TU.BK | Train : 0.6043 | Test : 0.5981
0.00000 0.00000 nan 0.00000


[*********************100%%**********************]  1 of 1 completed


VGI.BK | Train : 0.6212 | Test : 0.6371
0.00000 0.00000 nan 0.00000


[*********************100%%**********************]  1 of 1 completed


WHA.BK | Train : 0.6204 | Test : 0.5805
0.00000 0.00000 nan 0.00000


In [9]:
all_data.dropna(inplace=True)

In [10]:
all_data

,Index Name,Annual Return,Cumulative Returns,Annual Volatility,Sharpe Ratio,Sortino Ratio,Max Drawdown,Calmar Ratio,1_Train,1_Test,Test Date
^SET.BK,SET Index,0.065774,0.382045,0.120973,0.586892,0.889116,-0.207349,0.317217,0.605386,0.505074,2018-09-13 00:00:00
AEONTS.BK,AEON Thana Sinsap (Thailand) Public Company Li...,0.145183,0.758242,0.349984,0.561368,0.893914,-0.416000,0.348998,0.676828,0.594286,2019-09-02 00:00:00
AMATA.BK,Amata Corporation Public Company Limited,0.264899,1.884443,0.315083,0.902271,1.436620,-0.301543,0.878479,0.610207,0.584872,2019-04-23 00:00:00
AOT.BK,Airports of Thailand Public Company Limited,0.067051,0.271277,0.281656,0.367652,0.616628,-0.274074,0.244645,0.585209,0.584137,2020-02-21 00:00:00
BANPU.BK,Banpu Public Company Limited,0.122534,0.695447,0.384103,0.491838,0.753959,-0.577181,0.212297,0.600869,0.606771,2019-03-28 00:00:00
BBL.BK,Bangkok Bank Public Company Limited,0.134337,0.778409,0.235542,0.652367,0.998325,-0.294922,0.455499,0.586319,0.574653,2019-03-28 00:00:00
BDMS.BK,Bangkok Dusit Medical Services Public Company ...,0.110786,0.558989,0.215282,0.594837,0.931274,-0.255319,0.433912,0.631307,0.621013,2019-08-08 00:00:00
BH.BK,Bumrungrad Hospital Public Company Limited,0.216685,1.114286,0.237408,0.943897,1.555234,-0.205224,1.055845,0.582489,0.575286,2020-01-09 00:00:00
COM7.BK,Com7 Public Company Limited,-0.200527,-0.278788,0.339963,-0.485130,-0.642010,-0.394444,-0.508377,0.603121,0.609756,2022-06-27 00:00:00
DELTA.BK,Delta Electronics (Thailand) Public Company Li...,1.007320,23.109588,0.694838,1.342341,2.256692,-0.619792,1.625256,0.586319,0.519965,2019-03-28 00:00:00


In [11]:
all_data.to_excel("Trend_SET50.xlsx", index=True)